In [112]:
import os
tavily_api_key = os.getenv("TAVILY_API_KEY") 


In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults


In [3]:
search = TavilySearchResults(max_results=2)

In [4]:
search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1723299760, 'localtime': '2024-08-10 07:22'}, 'current': {'last_updated_epoch': 1723299300, 'last_updated': '2024-08-10 07:15', 'temp_c': 13.4, 'temp_f': 56.0, 'is_day': 1, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 4.9, 'wind_kph': 7.9, 'wind_degree': 236, 'wind_dir': 'WSW', 'pressure_mb': 1014.0, 'pressure_in': 29.95, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 90, 'cloud': 34, 'feelslike_c': 13.0, 'feelslike_f': 55.5, 'windchill_c': 13.0, 'windchill_f': 55.5, 'heatindex_c': 13.4, 'heatindex_f': 56.0, 'dewpoint_c': 11.4, 'dewpoint_f': 52.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 4.0, 'gust_mph': 7.6, 'gust_kph': 12.2}}"},
 {'url': 'https://forecast.weather.gov/zipcit

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
from langchain.tools.retriever import create_retriever_tool

In [7]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [8]:
tools = [search, retriever_tool]

In [14]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [15]:
model_with_tools = model.bind_tools(tools)

In [16]:
from langchain_core.messages import AIMessage,SystemMessage,HumanMessage
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [17]:
response = model_with_tools.invoke([HumanMessage(content="What's the political situation in bangladesh?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'political situation in Bangladesh'}, 'id': 'call_LEMDYBSKxBC28tgrHN0OsEbs', 'type': 'tool_call'}]


In [19]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [20]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)

In [21]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [22]:
agent_executor.invoke({"input": "hi!"})

{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [23]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

{'input': 'how can langsmith help with testing?',
 'output': "LangSmith is a platform for building production-grade LLM (Language Model) applications that can help with testing in the following ways:\n\n1. **Closely Monitor and Evaluate Applications**: LangSmith allows you to closely monitor and evaluate your application during the testing phase. This helps ensure that your application functions correctly and meets the required standards.\n\n2. **Ship Quickly and With Confidence**: By using LangSmith for testing, you can ship your applications quickly and with confidence, knowing that they have been thoroughly evaluated and monitored.\n\n3. **Trace Logging**: LangSmith provides multiple ways to log traces, allowing you to track the behavior of your application during testing. This trace logging can be instrumental in identifying issues and improving the application's performance.\n\n4. **Evaluation Framework**: LangSmith offers an evaluation framework that enables you to define test ca

In [28]:
response = agent_executor.invoke({"input": "tell me about the student protests in bangladesh"})

In [29]:
print(response['output'])

The student protests in Bangladesh have been significant, with young student demonstrators leading a movement against a longstanding leader in the country. The protests have been dubbed a "Gen Z revolution" and have involved a considerable number of young individuals advocating for change. The demonstrations initially started peacefully but escalated into violence, resulting in over 200 casualties. The students were protesting against the country's quota system for government jobs, which reserves 30% of positions for families of veterans. The protests have garnered attention both nationally and internationally. If you would like more detailed information, I can provide further insights from the sources I found.


In [9]:
from langchain.agents import (create_sql_agent,
                              create_json_agent,
                              create_react_agent,
                              create_structured_chat_agent,
                              create_vectorstore_agent,
                              create_vectorstore_router_agent,
                              AgentType)
from langchain_community.agent_toolkits import JsonToolkit
from langchain_community.tools.json.tool import JsonSpec
from langchain_experimental.agents.agent_toolkits import create_python_agent,create_pandas_dataframe_agent,create_csv_agent
from langchain.agents import initialize_agent

In [69]:
from langchain_groq import ChatGroq
import yaml
    
with open("data/endpoints.yml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
json_spec = JsonSpec(dict_=data, max_value_length=4000)


json_agent_executor = create_json_agent(
    llm=ChatOpenAI(model='gpt-4o-mini'), toolkit=JsonToolkit(spec=json_spec), verbose=True,handle_parsing_errors=True
)

In [71]:
response = json_agent_executor.invoke(
    "give me the endpoint to retrive all models\nonly provide the link and nothing else"
)



> Entering new AgentExecutor chain...
Action: json_spec_list_keys  
Action Input: data  
Observation: ['endpoints']
Thought:I need to explore the "endpoints" key to find the specific endpoint for retrieving all models.  
Action: json_spec_list_keys  
Action Input: data["endpoints"]  
Observation: ['completions', 'chat_completions', 'edits', 'embeddings', 'images_generations', 'images_edits', 'images_variations', 'files_upload', 'files_list', 'files_delete', 'fine_tunes_create', 'fine_tunes_list', 'fine_tunes_retrieve', 'fine_tunes_cancel', 'models_list', 'models_retrieve', 'models_delete', 'moderations']
Thought:I see that there is a key for 'models_list', which likely corresponds to the endpoint for retrieving all models. I will check the specific endpoint for 'models_list'.  
Action: json_spec_get_value  
Action Input: data["endpoints"]["models_list"]  
Observation: {'url': 'https://api.openai.com/v1/models', 'method': 'GET', 'description': 'List all available models.'}
Thought:I n

In [73]:
print(response['output'])

https://api.openai.com/v1/models


In [74]:
tools = [TavilySearchResults(max_results=1)]
prompt = hub.pull("hwchase17/structured-chat-agent")


In [77]:
print(prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format

In [106]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106")
agent = create_structured_chat_agent(llm, tools, prompt) 

In [79]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [80]:
response = agent_executor.invoke({"input": "how good is the current model released by meta?"})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": {"query": "Meta current model performance review"}
}
```[{'url': 'https://pubmed.ncbi.nlm.nih.gov/28057641/', 'content': 'A guide to systematic review and meta-analysis of prediction model performance. BMJ. 2017 Jan 5:356:i6460. doi: 10.1136/bmj.i6460.'}]{
  "action": "tavily_search_results_json",
  "action_input": {"query": "Meta current model performance review"}
}[{'url': 'https://decrypt.co/227263/meta-llama-3-open-source-ai-llm-review', 'content': 'Apr 19, 2024. 7 min read. Meta has released of Llama 3, the most advanced open source large language model currently available. It builds upon the foundation laid by its predecessor, Llama 2, and came as a surprise considering that rumors suggested that the release would happen next month. With its open-source roots, Llama-2 was instrumental in ...'}]{
  "action": "Final Answer",
  "action_input": "Meta has released Llama 3,

In [84]:
print(response['output'])

Meta has released Llama 3, which is described as the most advanced open source large language model currently available. It builds upon the foundation laid by its predecessor, Llama 2.


In [86]:
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
response = agent_executor.invoke({"input": "what is the sum of 1/2 + 1/4 + 1/8 + 1/16...?"})




> Entering new AgentExecutor chain...
I should use the search engine to find the sum of this infinite series.
Action: tavily_search_results_json
Action Input: "sum of 1/2 + 1/4 + 1/8 + 1/16..."[{'url': 'https://www.wolframalpha.com/input/?i=1/2+++1/4+++1/8+++1/16+++...', 'content': "1/2 + 1/4 + 1/8 + 1/16 + ... Natural Language. Math Input. Extended KeyboardExamplesUploadRandom. Compute answers using Wolfram's breakthrough technology & knowledgebase, relied on by millions of students & professionals. For math, science, nutrition, history, geography, engineering, mathematics, linguistics, sports, finance, music…."}]I should click on the link to find the answer.
Action: tavily_search_results_json
Action Input: "click on the link"[{'url': 'https://english.stackexchange.com/questions/53794/when-to-use-click-and-when-to-use-click-on', 'content': '"Click the link" would be OK, you are assuming that it is understood the mouse will be used to click the link, and are thus extending the proper

In [89]:
print(response['output'])

The sum of 1/2 + 1/4 + 1/8 + 1/16... is 1.


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

tools = [get_word_length]

In [124]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [125]:
llm_with_tools = llm.bind_tools(tools)

In [126]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [127]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [129]:
list(agent_executor.stream({"input": "How many letters in the word INFORMATION"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'INFORMATION'}`


11There are 11 letters in the word "INFORMATION".

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'INFORMATION'}, log="\nInvoking: `get_word_length` with `{'word': 'INFORMATION'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Shr6LjddwBHnAL4kNG9c4JW5', 'function': {'arguments': '{"word":"INFORMATION"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-8cdd7ff9-d21f-4e26-b840-9f495f0bcb9c', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'INFORMATION'}, 'id': 'call_Shr6LjddwBHnAL4kNG9c4JW5', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"INFORMATION"}', 'id': 'call_Shr6LjddwBHnAL4kNG9c4JW5', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_Shr6LjddwBHnAL4kNG9c4JW5')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Shr6LjddwBHnAL4kNG9c4JW5', 

In [1]:
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma

llm = ChatOpenAI(model = 'gpt-3.5-turbo', temperature=0)

c:\Users\aksha\anaconda2\envs\nlp\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./state_of_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  
)

c:\Users\aksha\anaconda2\envs\nlp\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
c:\Users\aksha\anaconda2\envs\nlp\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [3]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(texts))]

vector_store.add_documents(documents=texts, ids=uuids)

['0b43f895-e235-47ed-a2bf-bcba81370728',
 'a9ad805c-baab-4be5-906c-1f33cc4097ca',
 '463c1e67-fcd2-4f49-85b4-646fd2b1a91a',
 '6e59dd3e-6918-414f-91de-15c0ad88a74b',
 '08c3b23e-82fa-4245-8a69-3c68019f66f8']

In [6]:
from langchain.agents.agent_toolkits import (
    VectorStoreInfo,
    VectorStoreToolkit,
    create_vectorstore_agent,
)

vectorstore_info = VectorStoreInfo(
    name="state_of_union_address",
    description="the most recent state of the Union address",
    vectorstore=vector_store,
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info,llm=llm)
agent_executor = create_vectorstore_agent(llm=llm, toolkit=toolkit, verbose=True)

In [8]:
agent_executor.run(
    "What happened in 1941 in union address?"
)



> Entering new AgentExecutor chain...
I should use the state_of_union_address tool to find information about the 1941 State of the Union address.
Action: state_of_union_address
Action Input: 1941
Observation: In 1941, Franklin Roosevelt spoke about the now-famous four freedoms during his State of the Union message.
Thought:I now know the final answer
Final Answer: In 1941, Franklin Roosevelt spoke about the now-famous four freedoms during his State of the Union message.

> Finished chain.


'In 1941, Franklin Roosevelt spoke about the now-famous four freedoms during his State of the Union message.'

In [6]:
from langchain_experimental.tools.python.tool import PythonREPLTool

agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [7]:
input_list=["Yash",17,"Akul",20,"Tinku",30]


In [8]:
response = agent.run(f"""Given the input list {input_list}, convert it \
into a dictionary where the keys are the names occuring at even indices and \
the values are the roll number occuring at odd indices""")

c:\Users\aksha\anaconda2\envs\nlp\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


We can iterate through the list and create a dictionary by pairing elements at even indices with elements at odd indices.

Action: Python_REPL
Action Input: student_list = ['Yash', 17, 'Akul', 20, 'Tinku', 30]
Observation: 
Thought:Let's create a dictionary by pairing elements at even indices with elements at odd indices.

Action: Python_REPL
Action Input: student_dict = {student_list[i]: student_list[i+1] for i in range(0, len(student_list), 2)}
print(student_dict)
Observation: {'Yash': 17, 'Akul': 20, 'Tinku': 30}

Thought:I now know the final answer

Final Answer: {'Yash': 17, 'Akul': 20, 'Tinku': 30}

> Finished chain.


In [9]:
print(response)

{'Yash': 17, 'Akul': 20, 'Tinku': 30}


In [28]:
response = agent.run(f"""Write a function in Python to perform BFS search in the game missionaries and cannibals\nReturn the python function""")




> Entering new AgentExecutor chain...
I can write a Python function to perform BFS search in the game missionaries and cannibals.
Action: Python_REPL
Action Input:
```python
def bfs_missionaries_cannibals():
    start = (3, 3, 1)  # initial state (3 missionaries, 3 cannibals, 1 boat on the starting side)
    goal = (0, 0, 0)   # goal state (0 missionaries, 0 cannibals, 0 boat on the starting side)
    
    # Implement BFS algorithm here
    
    return path_to_goal
```
Observation: 
Thought:I need to implement the BFS algorithm inside the function.
Action: I will write the BFS algorithm inside the function.
Action Input:
```python
from collections import deque

def bfs_missionaries_cannibals():
    start = (3, 3, 1)  # initial state (3 missionaries, 3 cannibals, 1 boat on the starting side)
    goal = (0, 0, 0)   # goal state (0 missionaries, 0 cannibals, 0 boat on the starting side)
    
    queue = deque([(start, [])])
    visited = set()
    
    while queue:
        state, path =

In [1]:
import pandas as pd

df = pd.read_csv('data/titanic.csv')

In [7]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
agent = create_pandas_dataframe_agent(ChatGroq(temperature=0, model="llama-3.1-8B-instant"),
    df,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True)

In [10]:
agent.invoke("what is the ratio of the people who survived and the people who didn't?")



> Entering new AgentExecutor chain...
Thought: To find the ratio of people who survived and those who didn't, I need to count the number of survivors and non-survivors in the dataframe.
Action: python_repl_ast
Action Input: `df['Survived'].value_counts()`Survived
0    549
1    342
Name: count, dtype: int64Thought: Now that I have the count of survivors and non-survivors, I can calculate the ratio by dividing the count of survivors by the count of non-survivors.
Action: python_repl_ast
Action Input: `df['Survived'].value_counts()[1] / df['Survived'].value_counts()[0]`0.6229508196721312Thought: Now that I have the ratio of survivors to non-survivors, I can express it as a percentage by multiplying it by 100.
Action: python_repl_ast
Action Input: `df['Survived'].value_counts()[1] / df['Survived'].value_counts()[0] * 100`62.295081967213115Thought: I now know the final answer
Final Answer: 62.295081967213115

> Finished chain.


{'input': "what is the ratio of the people who survived and the people who didn't?",
 'output': '62.295081967213115'}

In [18]:
agent = create_csv_agent(
    ChatOpenAI(model='gpt-3.5-turbo'),
    "data/titanic.csv",
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    allow_dangerous_code = True
)

c:\Users\aksha\anaconda2\envs\nlp\Lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [19]:
agent.invoke('what is the average fee paid by the passengers')



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Fare'].mean()"}`


32.204207968574636The average fee paid by the passengers is approximately $32.20.

> Finished chain.


{'input': 'what is the average fee paid by the passengers',
 'output': 'The average fee paid by the passengers is approximately $32.20.'}

In [5]:
from langchain.utilities import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

db = SQLDatabase.from_uri("sqlite:///MyFirstDatabase.db")


In [6]:
db.get_usable_table_names()


['TITANIC']

In [12]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
llm = ChatGroq(model='llama-3.1-8B-instant')
model = ChatOpenAI(model='gpt-3.5-turbo')

In [13]:
agent_executor = create_sql_agent(
    llm=model,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code = True
)

In [16]:
response = agent_executor.run(
    "What is the average fee paid by the people?\nGive me the SQL command used to calculate this"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


TITANIC
Invoking: `sql_db_schema` with `{'table_names': 'TITANIC'}`



CREATE TABLE "TITANIC" (
	"PassengerId" TEXT, 
	"Survived" TEXT, 
	"Pclass" TEXT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" TEXT, 
	"SibSp" TEXT, 
	"Parch" TEXT, 
	"Ticket" TEXT, 
	"Fare" TEXT, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
3 rows from TITANIC table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
1	0	3	Braund, Mr. Owen Harris	male	22	1	0	A/5 21171	7.25		S
2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38	1	0	PC 17599	71.2833	C85	C
3	1	3	Heikkinen, Miss. Laina	female	26	0	0	STON/O2. 3101282	7.925		S
*/
Invoking: `sql_db_query` with `{'query': 'SELECT AVG(Fare) AS AverageFee FROM TITANIC'}`


[(32.204207968574636,)]The average fee paid by the people is approximately $32.20. 

The SQL command used to calculate this is:
```sql
SELECT AVG(Fare) AS AverageFee FROM TITANIC
```

> 

In [17]:
print(response)

The average fee paid by the people is approximately $32.20. 

The SQL command used to calculate this is:
```sql
SELECT AVG(Fare) AS AverageFee FROM TITANIC
```
